# (some) Related Work 
- A spectral Monte Carlo method for the Poisson equation
- Chebyshev Greeks Smoothing Gamma without Bias
- Approximation Theory and Approximation Practice (Lloyd N. Trefethen)
- Regression-based Monte Carlo Integration

We haven't looked into the literature for random ODEs (random start and coefficients not stochastic differential equations)



# Abstract
(we will write this later)

# Russian Roulette Modded
In period1 we missed a simple insight of Russian Roulette. Instead of killing of simulations completely you can substitute an approximation instead again without changing the expectance. This shouldn't be confused with control variates although both need approximations of the solution. <br>

To get our example in the code to work we did some algebra to figure out the modification:

In [83]:
from random import random
from math import exp

def approx_sol(t):
    return 1+t    

def Y(t):
    k = t if t<1 else 1
    return (1+t*Y(random()*t)-(1-k)*approx_sol(t))/k if random()<k else approx_sol(t)

sol = 0
nsim = 10**3
t = 3

for _ in range(nsim):
    sol += Y(t)/nsim

percentage_error = (sol - exp(t))/exp(t)

print(f"exp({t}) is approx = {sol}")
print(f"%error = {percentage_error}")

exp(3) is approx = 18.175346900390753
%error = -0.09510276126060158


# Random ODEs
Let's get a quick example in (and maybe work out more later):
$$
y' = Ay, y(0)=1 , A = \text{Uniform}(0,1) 
$$
With solution $y(t)= e^{tA}$ which is a random variable. With the techniques we have right now we can simulate unbiased estimates of simulations of the solution (yeah randomness on randomness).With these you can get "weak" solutions ($E_y[f(y(t))]$ with $f$ analytic) to the problem: 
$$
\begin{align*}
E_y[f(y(t))] &= E_y[E_A[f(y(t)) \mid  A]] \\    
           &= E_y[E_A[f(y(t,A))]] \\    
            &= E_y[E_A[f(E_Y[Y(t,A)])]] \\
\end{align*}
$$
remember how we dealt with $f(E_Y[Y(t,A)])$ in period1? This makes things like unbiased estimators of the expectance and variance possible which can be used to get confidence intervals using Chebychev's inequality (https://en.wikipedia.org/wiki/Chebyshev%27s_inequality). <br>

In our example we can find the following by hand:
$$
\begin{align*}
E_A[y(t)] &=  \frac{e^{t}}{t} - \frac{1}{t} \\ 
E_A[y^{2}(t)] &=  \frac{e^{2t}}{2t} - \frac{1}{2t}  
\end{align*}
$$
In code all this looks like this:

In [105]:
from random import random
from math import exp,sqrt

#estimator for y(t,a)
def Ya(t,a):
    return (1+a*Ya(random()*t,a) if random()<t else 1) if t<1 else 1+t*a*Ya(random()*t,a)

# estimator for y(t)
def Y(t):
    return Ya(t,random())
    
# estimator for y(t)**2
def Y2(t):
    A = random()
    return Ya(t,A)*Ya(t,A) 

sol = 0
sol2 = 0
nsim = 10**4
t = 3 # 0<t

for _ in range(nsim):
    sol += Y(t)/nsim
    sol2 += Y2(t)/nsim

s =  exp(t)/t -1/t #analytic solution
percentage_error = (sol - s)/s

s2 =  exp(2*t)/(2*t) -1/(2*t) #analytic solution
percentage_error2 = (sol2 - s2)/s2

print(f"E(y({t}))   is approx {sol},",f"%error = {percentage_error}")
print(f"E(y^2({t})) is approx {sol2},",f"%error = {percentage_error2}")

sd = sqrt(sol2-sol**2)
print(f"standard deviation of y(3) is approx (biased) {sd}")

E(y(3))   is approx 6.459277460154641, %error = 0.015315024065219401
E(y^2(3)) is approx 62.83166864932534, %error = -0.06321312492576975
standard deviation of y(3) is approx (biased) 4.594497071722166
